# Arts Council Funding by Org Type

This script combines funding information from the Arts Council Investment Programme, National Lottery Project Grants and Companies House data to explore the relationship between organisation size and the amount of funding received. 

Note: As a proxy for organisation size, we have used the accounting category from Companies House. Accounts categories offer an indication of the size and type of organisation. For definitions of account types, <a href="https://find-and-update.company-information.service.gov.uk/guides/accounts/chooser">see the Companies House definitions</a>.  



In [1]:
# Import libraries

import pandas as pd
from pathlib import Path
import os

In [2]:
# Define paths to the files we need

OUT_DIR = Path('../src/insight/funding/_data')
OUT_DIR.mkdir(exist_ok=True, parents=True)

ORG_DATA = Path('../data/culture_landscape.csv')
INVESTMENT_DATA = Path('../raw/arts-council-investment-programme.csv')
GRANTS_DATA = Path('../raw/arts-council-project-grants.csv')
CORRECTIONS = Path('../raw/landscape-map-corrections.csv')

In [3]:
# Read in the files we need

# Only either funding targetted at Newcastle, or organisations based in Newcastle
org_data = pd.read_csv(ORG_DATA).query('(funding_geo_code == "E08000021") or (oslaua == "E08000021")')
corrections = pd.read_csv(CORRECTIONS).set_index('organisation').match.to_dict()
invest_data = pd.read_csv(INVESTMENT_DATA).dropna(subset=['Applicant Name']).replace({'Applicant Name': corrections}).query('`Local authority` == "Newcastle upon Tyne"')
grants_data = pd.read_csv(GRANTS_DATA).replace({'Recipient': corrections})

In [4]:
# TODO: We may wish to present the number of UNKNOWN categories. 
# TODO: Fuzzy match and include list of corrections. 

# Drop records where the accounts catgory is unknown. 
acc_cat = (
    org_data
        .dropna(subset=['accounts_category'])
        # .query('~accounts_category.isin(["DORMANT"])')
        .loc[:, ['organisation', 'accounts_category']]
)

# Merge in Project Grants data
funding_amounts = (
    acc_cat.merge(grants_data[['Recipient', 'Award amount']], 
                          left_on='organisation', 
                          right_on='Recipient', 
                          how='left')
    .rename(columns={'Award amount': 'project_grant_amount'})
    .drop(columns='Recipient')
) 

# Merge in investment grants data
funding_amounts = (
    funding_amounts.merge(invest_data[['Applicant Name', '2023-26 Annual Funding (Offered 4 Nov 2022)']], 
                          left_on='organisation', 
                          right_on='Applicant Name', 
                          how='left')
    .rename(columns={'2023-26 Annual Funding (Offered 4 Nov 2022)': 'investment_amount'})
    .drop(columns='Applicant Name')
)


In [ ]:
funding_amounts

In [6]:
funding_amounts['total_funding'] = (funding_amounts['project_grant_amount'].fillna(0) + 
                                    funding_amounts['investment_amount'].fillna(0)).astype(int)

total_funding = funding_amounts[['organisation', 'accounts_category', 'total_funding']]

In [7]:
# Group by accounting category and calculate the average funding amount

average_grant_award_by_acc_cat = (
    total_funding.groupby('accounts_category')['total_funding'].mean()
    .astype(int)
    .sort_values(ascending=False)
    .to_csv(os.path.join(OUT_DIR, 'grant_by_org_size.csv'))
)